In [2]:
# Step 2: Import necessary libraries
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
import os
import warnings

# Suppress all Python warnings
warnings.filterwarnings('ignore')

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all logs, 1 = filter out INFO, 2 = filter out INFO and WARNING, 3 = ERROR only

Explanation
This code imports the necessary libraries:

- keras_tuner: Used for hyperparameter tuning.
- Sequential: A linear stack of layers in Keras.
- Dense, Flatten: Common Keras layers.
- mnist: The MNIST dataset, a standard dataset for image classification.
- Adam: An optimizer in Keras.

In [3]:

# Step 3: Load and preprocess the MNIST dataset
(x_train, y_train), (x_val, y_val) = mnist.load_data()
x_train, x_val = x_train / 255.0, x_val / 255.0

print(f'Training data shape: {x_train.shape}')
print(f'Validation data shape: {x_val.shape}')

Training data shape: (60000, 28, 28)
Validation data shape: (10000, 28, 28)


### Exercise 2: Defining the model with hyperparameters
In this exercise, you define a model-building function that uses the HyperParameters object to specify the number of units in a dense layer and the learning rate. This function returns a compiled Keras model that is ready for hyperparameter tuning.

Define a model-building function:

- Create a function build_model that takes a HyperParameters object as input.
- Use the HyperParameters object to define the number of units in a dense layer and the learning rate for the optimizer.
- Compile the model with sparse categorical cross-entropy loss and Adam optimizer.

In [4]:
# Define a model-building function 

def build_model(hp):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


### Exercise 3: Configuring the hyperparameter search
This exercise guides you through configuring Keras Tuner. You create a RandomSearch tuner, specifying the model-building function, the optimization objective, the number of trials, and the directory for storing results. The search space summary provides an overview of the hyperparameters being tuned.

Create a RandomSearch Tuner:

- Use the RandomSearch class from Keras Tuner.
- Specify the model-building function, optimization objective (validation accuracy), number of trials, and directory for storing results.

In [5]:
# Create a RandomSearch Tuner 

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='intro_to_kt'
)

# Display a summary of the search space 
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


Explanation
This code sets up a Keras Tuner RandomSearch:

- build_model: The model-building function.
- objective='val_accuracy': The metric to optimize (validation accuracy).
- max_trials=10: The maximum number of different hyperparameter configurations to try.
- executions_per_trial=2: The number of times to run each configuration.
- directory='my_dir': Directory to save the results.
- project_name='intro_to_kt': Name of the project for organizing results.
Displays a summary of the hyperparameter search space, providing an overview of the hyperparameters being tuned.

### Exercise 4: Running the hyperparameter search
In this exercise, you run the hyperparameter search using the search method of the tuner. You provide the training and validation data along with the number of epochs. After the search is complete, the results summary displays the best hyperparameter configurations found.

Run the search:

- Use the search method of the tuner.
- Pass in the training data, validation data, and the number of epochs

In [6]:
# Run the hyperparameter search 
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val)) 

# Display a summary of the results 
tuner.results_summary() 

Trial 10 Complete [00h 01m 50s]
val_accuracy: 0.9796499907970428

Best val_accuracy So Far: 0.9808500111103058
Total elapsed time: 00h 16m 26s
Results summary
Results in my_dir\intro_to_kt
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 05 summary
Hyperparameters:
units: 480
learning_rate: 0.0013263444829043885
Score: 0.9808500111103058

Trial 09 summary
Hyperparameters:
units: 416
learning_rate: 0.0025214364578715043
Score: 0.9796499907970428

Trial 03 summary
Hyperparameters:
units: 384
learning_rate: 0.001561164109127508
Score: 0.9781999886035919

Trial 07 summary
Hyperparameters:
units: 160
learning_rate: 0.0012564412521369585
Score: 0.9772499799728394

Trial 00 summary
Hyperparameters:
units: 160
learning_rate: 0.0005162885734768005
Score: 0.976500004529953

Trial 02 summary
Hyperparameters:
units: 160
learning_rate: 0.0007950738895236766
Score: 0.976500004529953

Trial 01 summary
Hyperparameters:
units: 224
learning_rate: 0.00220845836973479
Score: 0

### Exercise 5: Analyzing and using the best hyperparameters
In this exercise, you retrieve the best hyperparameters found during the search and print their values. You then build a model with these optimized hyperparameters and train it on the full training data set. Finally, you evaluate the model’s performance on the test set to ensure that it performs well with the selected hyperparameters.

Retrieve the best hyperparameters:

- Use the get_best_hyperparameters method to get the best hyperparameters.
- Print the optimal values for the hyperparameters.
Build and train the model:

- Build a model using the best hyperparameters.
- Train the model on the full training data set and evaluate its performance on the test set.